In [19]:
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.applications import ResNet50V2

tf.__version__

'2.10.0'

In [3]:
PATH = 'data'

os.listdir(PATH)

['archive (12).zip',
 'jpeg224',
 'sample_submission.csv',
 'test.csv',
 'train.csv',
 'train_split.csv',
 'val_split.csv',
 'subset.csv']

In [26]:
train = pd.read_csv(f'{PATH}/subset.csv')

val = pd.read_csv(f'{PATH}/val_split.csv')

test = pd.read_csv(f'{PATH}/test.csv')

train.shape, val.shape, test.shape

((2220, 8), (10932, 8), (10982, 5))

In [6]:
train.head()

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_0533349,IP_5208504,female,45.0,lower extremity,unknown,benign,0
1,ISIC_8814612,IP_0414408,male,50.0,torso,unknown,benign,0
2,ISIC_6515241,IP_6245507,male,45.0,lower extremity,unknown,benign,0
3,ISIC_5075261,IP_2117218,male,40.0,upper extremity,unknown,benign,0
4,ISIC_2624460,IP_1969685,male,50.0,torso,unknown,benign,0


In [16]:
BATCH_SIZE = 64

def decode(name, label):
    img = tf.io.read_file(name)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) 
    return img, label


def load_ds(df):
    imgs, labels = df["image_name"].values, df["target"].values
    imgs = [f'{PATH}/jpeg224//train/{name}.jpg' for name in imgs]
    ds = tf.data.Dataset.from_tensor_slices((imgs, labels))
    ds = ds.map(decode)
    ds = ds.shuffle(2048)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [17]:
train_ds = load_ds(train)
val_ds = load_ds(val)

In [18]:
for img, label in train_ds.take(1):
    print(img)
    print(label)

tf.Tensor(
[[[[108.  91.  71.]
   [111.  94.  74.]
   [113.  96.  76.]
   ...
   [134. 112.  98.]
   [127. 107. 100.]
   [ 78.  57.  54.]]

  [[140. 120.  96.]
   [140. 122.  98.]
   [145. 125. 101.]
   ...
   [170. 149. 130.]
   [165. 143. 132.]
   [113.  93.  86.]]

  [[131. 105.  78.]
   [133. 109.  81.]
   [138. 112.  85.]
   ...
   [153. 131. 107.]
   [146. 125. 106.]
   [ 97.  75.  61.]]

  ...

  [[136. 102.  77.]
   [136. 105.  77.]
   [141. 107.  79.]
   ...
   [170. 148. 124.]
   [158. 141. 121.]
   [107.  94.  77.]]

  [[142. 114.  93.]
   [143. 116.  95.]
   [145. 117.  95.]
   ...
   [168. 151. 131.]
   [163. 151. 137.]
   [121. 113. 102.]]

  [[109.  83.  66.]
   [110.  84.  67.]
   [113.  88.  68.]
   ...
   [ 34.  21.   4.]
   [ 41.  31.  21.]
   [  8.   1.   0.]]]


 [[[147.  97.  88.]
   [152. 102.  93.]
   [159. 106.  98.]
   ...
   [181. 131. 134.]
   [180. 129. 134.]
   [178. 127. 132.]]

  [[142.  92.  83.]
   [148.  98.  89.]
   [157. 104.  96.]
   ...
   [182. 1

In [23]:
IMAGE_SIZE = (224, 224, 3)

encoder = ResNet50V2(
    include_top=False,
    input_shape=IMAGE_SIZE,
    weights='imagenet'
)

encoder.trainable = False

inputs = tf.keras.Input(shape=IMAGE_SIZE)
x = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = encoder(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling_3 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 resnet50v2 (Functional)     (None, 7, 7, 2048)        23564800  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,566,849
Trainable params: 2,049
Non-trainable params: 23,564,800
_____________________________________________

In [24]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[keras.metrics.AUC(name='auc')]
)

In [25]:
model.fit(train_ds, epochs=10, validation_data=val_ds, validation_steps=10)

Epoch 1/10
35/35 [==============================] - 140s 4s/step - loss: 0.2305 - auc: 0.4503 - val_loss: 0.1564 - val_auc: 0.3338
Epoch 2/10
35/35 [==============================] - 118s 3s/step - loss: 0.1111 - auc: 0.4411 - val_loss: 0.1103 - val_auc: 0.5346
Epoch 3/10
35/35 [==============================] - 122s 3s/step - loss: 0.0949 - auc: 0.5333 - val_loss: 0.1117 - val_auc: 0.5523
Epoch 4/10
35/35 [==============================] - 127s 3s/step - loss: 0.0848 - auc: 0.6915 - val_loss: 0.1193 - val_auc: 0.6651
Epoch 5/10
35/35 [==============================] - 128s 3s/step - loss: 0.0782 - auc: 0.7764 - val_loss: 0.0880 - val_auc: 0.6649
Epoch 6/10
35/35 [==============================] - 116s 3s/step - loss: 0.0735 - auc: 0.8093 - val_loss: 0.1138 - val_auc: 0.8041
Epoch 7/10
35/35 [==============================] - 119s 3s/step - loss: 0.0697 - auc: 0.8503 - val_loss: 0.0970 - val_auc: 0.6477
Epoch 8/10
35/35 [==============================] - 115s 3s/step - loss: 0.0659 - a

In [27]:
BATCH_SIZE = 256

def decode_test(name):
    img = tf.io.read_file(name)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.cast(img, tf.float32) 
    return img


def load_test_ds(df):
    imgs = df["image_name"].values
    imgs = [f'{PATH}/jpeg224/test/{name}.jpg' for name in imgs]
    ds = tf.data.Dataset.from_tensor_slices(imgs)
    ds = ds.map(decode_test)
    ds = ds.batch(BATCH_SIZE)
    return ds

In [28]:
test_ds = load_test_ds(test)

In [29]:
preds = model.predict(test_ds)

43/43 [==============================] - 384s 9s/step


In [30]:
preds

array([[0.00243396],
       [0.00333899],
       [0.01711598],
       ...,
       [0.01144026],
       [0.00055865],
       [0.09401157]], dtype=float32)

In [31]:
submission = pd.DataFrame({
    'image_name': test['image_name'].values,
    'target': preds.ravel()
})

submission.head()

,image_name,target
0,ISIC_0052060,0.002434
1,ISIC_0052349,0.003339
2,ISIC_0058510,0.017116
3,ISIC_0073313,0.000691
4,ISIC_0073502,0.008137


In [32]:
submission.to_csv('submission.csv', index=False)